In [3]:
%%writefile C:\Anaconda\Lib\CAMP.py
    
import datetime as dt
import pandas as pd
import numpy as np
import math as math 
import MathsUtilities as MUte
from scipy.optimize import curve_fit

CampConstants = {
    'k':-0.19,
    'VernalisationThreshold':1.0,
    'MethalationThreshold':1.0,
}

def LARPTQ(PTQ,maxLAR,minLAR,PTQhf):
    """Calculate the Leaf Appearance Rate at a given PTQ for a genotype of given minLAR and MaxLAR as formualted by Baumont etal 2019 Journal Expt Botany, Equation 3. 
    Args:
        PTQ: PhotoThermal Quatent (mmol PAR m-2 oCd-1)
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    """
    return minLAR+((maxLAR-minLAR)*PTQ)/ (PTQhf+PTQ)

def CalcPTQ(Tt,Pp,lightIntensity):
    Radn = Pp * 3600 * lightIntensity * 2  # Hours * s/h * MJ/m2/s * Coeff_PAR->Total 
    Ft = 1
    if Tt < 15:
        Ft = Tt/15
    return (Radn * 2.285 * Ft)/Tt

def CalcdHS(Tt,HS,PTQ,maxLAR,minLAR,PTQhf):
    """Calculate Haun stage
    Args:
        Tt: Thermal time increment
        HS: Current Haun Stage
        PTQ: PhotoThermalQuotient
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Value to increment Haun stage by
    """
    BasePhyllochron = 1/LARPTQ(PTQ,maxLAR,minLAR,PTQhf)
    StageFactor = np.interp(HS,[0,2,3,7,8,11,12],[1.0,1.0,1.0,1.0,1.4,1.4,1.4],False)
    Phyllochron = BasePhyllochron*StageFactor
    return Tt/Phyllochron

def CalcdPPVrn(Pp, baseUR, maxUR, dHS):
    """ Calculate delta of upregulation for photo period (Pp) sensitive genes
    
    Args:
        Pp: Photoperiod
        baseUR: dVrn/HS below 8h Pp
        maxUR: dVrn/HS above 16h Pp
        dHS: delta haun stage
    Returns:
        delta Vrn representing the additional expression of that gene today
    """
    if Pp <= 8.0:
        return baseUR * dHS
    if (Pp > 8.0) and (Pp < 16.0):
        return (baseUR + (maxUR * (Pp-8)/(16-8))) * dHS
    if (Pp >= 16.0):
        return maxUR * dHS  

def CalcBaseUpRegVrn1(Tt, dHS, BaseDVrn1):
    """ Calculate upregulation of base Vrn1
    
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        BaseDVrn1: coeffociennt for Base Vrn1 expression
        
    Returns:
        delta BaseVrn1 representing the additional Vrn1 expression from base expression
    """
    if Tt < 0: 
        BaseDVrn1 = 0
    return BaseDVrn1 * dHS

def CalcColdUpRegVrn1(Tt,dHS, MaxDVrn1, k):
    """ Upregulation of Vrn1 from cold.  Is additional to base vrn1
        BaseDVrn1 in seperate calculation otherwise te same as Brown etal 2013
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        MaxDVrn1: coefficient for Maximum upregulation of Vrn1
        
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    UdVrn1 = MaxDVrn1 * np.exp(k*Tt)
    if (Tt < 20):
        return UdVrn1 * dHS
    else:
        return -5

def TSHS(FLN):
    """Haun stage timing of terminal spikelet 
       Inverts equation 5 from Brown etal 2013 FLN =  2.85 + 1.1*TSHS
       Note the intercept differs, was typeo on publication
    Args:
        FLN: The final leaf number
    Returns:
        Estimation of the haun stage timing of terminal spikelet
    """
    return (FLN - 2.85)/1.1

def CAMPmodel(Out, Day, Tt, Pp, Params, Consts, TtEmerge, LightIntensity):
    """ The Cereal Anthesis Molecular Phenology model.
        Based on the ideas presented in Brown etal 2014 (Annals of Botany)
        Alterations made replacing Vrn4 notion with methalation of Vrn1, introducing VrnX to account for long day vernalisation
        Vrn2 approach changed to have initial high expression at emergence with up regulation rate following
        Other alterations to implement working code base
    Args:
        Out: "FLN" or None.  If "FLN" will only return estimated FLN else will return full dataframe with daily state variable values
        Day: List, 1:EndDay representing the timesteps in model run
        Tt: List of same length as Day, representing daily temperature
        Pp: List of same length as Day, representing daily photoperiod
        Params: Series with genotype parameters fitted by CalcCultivarVrnCoeffs() 
        k: Temperature response coefficient for Vrn1 response,
        VernSatThreshold: The amount of persistent Vrn1 that must be expressed to enable vernalisation,
        MethalationThreshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start,
        TtEmerge: Thermaltime from sowing to emergence
    """
    # Set up Data structure and initialise values
    DF = pd.DataFrame(index = [0], columns = ['Day','Tt','Pp'])
    IsGerminated = False
    IsEmerged = False
    IsMethalating = False
    IsVernalised = False
    IsInduced = False
    IsReproductive = False
    IsAtFlagLeaf = False
    DF.loc[0,'Day'] = 0
    DF.loc[0,'Tt'] = Tt[0]
    DF.loc[0,'Pp'] = Pp[0]
    DF.loc[0,'PTQ'] = 0
    DF.loc[0,'Stage'] = ""
    DF.loc[0,'HS'] = 0.0
    DF.loc[0,'LPpHS'] = 0.0
    DF.loc[0,'AccumTt'] = 0
    DF.loc[0,'BaseVrn1'] = 0
    DF.loc[0,'ColdVrn1'] = 0
    DF.loc[0,'MethColdVrn1'] = 0
    DF.loc[0,'Vrn1'] = 0
    DF.loc[0,'Vrn2'] = 0.0
    DF.loc[0,'Vrn3'] = 0.0
    DF.loc[0,'VrnX'] = 0.0
    DF.loc[0,'VSHS'] = 0
    DF.loc[0,'FIHS'] = 0
    DF.loc[0,'TSHS'] = 0
    DF.loc[0,'FLN'] = 2.86
    DF.loc[0,'dHS'] = 0
    DF.loc[0,'dLPpHS'] = 0
    DF.loc[0,'dBaseVrn1'] = 0
    DF.loc[0,'dColdVrn1'] = 0
    DF.loc[0,'dMethColdVrn1'] = 0
    DF.loc[0,'pVrn2'] = 0
    DF.loc[0,'dpVrn2'] = 0
    DF.loc[0,'dVrn3'] = 0
    DF.loc[0,'dVrnX'] = 0
    d = 1
    
    # Model daily loop
    while (IsAtFlagLeaf == False) and (d < Day[-1]):
        #Sow crop on day 1
        DF.loc[d,'Day'] = d
        if (IsGerminated == False):
            DF.loc[d,'Stage'] = 'Germination'
            IsGerminated = True
            GerminationDay = True
        # Set daily environment variables
        DF.loc[d,'Tt'] = Tt[d-1]
        DF.loc[d,'Pp'] = Pp[d-1]
        #Zero set deltas
        DF.loc[d,'dLPpHS'] = 0
        DF.loc[d,'dBaseVrn1'] = 0
        DF.loc[d,'dColdVrn1'] = 0
        DF.loc[d,'dMethColdVrn1'] = 0
        DF.loc[d,'pVrn2'] = 0
        DF.loc[d,'dpVrn2'] = 0
        #DF.loc[d,'PPdpVrn2'] = 0
        #DF.loc[d,'PPpVrn2'] = 0
        DF.loc[d,'dVrn2'] = 0
        DF.loc[d,'dVrn3'] = 0
        DF.loc[d,'dVrnX'] = 0
                
        DF.loc[d,'AccumTt'] = DF.loc[d-1,'AccumTt'] + DF.loc[d,'Tt']
        
        if (DF.loc[d,'AccumTt'] > TtEmerge) and (IsEmerged==False):
            IsEmerged = True
            DF.loc[d,'Stage'] = 'Emergence'
        
        DF.loc[d,'PTQ'] = 1#CalcPTQ(DF.loc[d,'Tt'],DF.loc[d,'Pp'],LightIntensity)
        if IsEmerged==True:
            DF.loc[d,'dHS'] = CalcdHS(DF.loc[d,'Tt'],DF.loc[d-1,'HS'],DF.loc[d,'PTQ'],Params.maxLAR,Params.minLAR,Params.PTQhf)
            DF.loc[d,'dLPpHS'] = DF.loc[d,'dHS'] * CalcdPPVrn(DF.loc[d,'Pp'], 0, 1, 1)
        else:
            DF.loc[d,'dHS']  = DF.loc[d,'Tt']/(1/LARPTQ(1.0,Params.maxLAR,Params.minLAR,Params.PTQhf))
            DF.loc[d,'dLPpHS'] = 0
       
        #increment HS
        if IsEmerged == True:
            DF.loc[d,'HS'] = DF.loc[d-1,'HS'] +  DF.loc[d,'dHS']
            DF.loc[d,'LPpHS'] = DF.loc[d-1,'LPpHS'] + DF.loc[d,'dLPpHS']
        else:
            DF.loc[d,'HS'] = 0
            DF.loc[d,'LPpHS'] = 0
    
        #Work out base, VrnX, cold induced Vrn1 expression and methalyation until vernalisation is complete
        if (IsVernalised==False): 
            DF.loc[d,'dBaseVrn1'] = CalcBaseUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.BaseDVrn1) 
            DF.loc[d,'dColdVrn1'] = CalcColdUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.MaxDVrn1,Consts['k'])
            DF.loc[d,'ColdVrn1'] = max(0.0,DF.loc[d-1,'ColdVrn1'] + DF.loc[d,'dColdVrn1'])
            
            #Work out if methalation of cold response has started 
            if DF.loc[d,'ColdVrn1']>= Consts['MethalationThreshold']:
                IsMethalating = True
            else:
                IsMethalating = False
                
            # If methalting calculate methVrn1
            if (IsMethalating == True):
                DF.loc[d,'dMethColdVrn1'] = min(DF.loc[d,'ColdVrn1'] - Consts['MethalationThreshold'],
                                                max(0,DF.loc[d,'dColdVrn1']))
            
            #Then work out Vrn2 and VrnX expression 
            DF.loc[d,'dVrnX'] = 0.0
            DF.loc[d,'dVrn2'] = 0.0
            if (IsEmerged==True):
                if DF.loc[d-1,'MethColdVrn1'] == 0.0:
                    DF.loc[d,'dVrnX'] = CalcdPPVrn(DF.loc[d,'Pp'], 0.0, Params.MaxDVrnX, DF.loc[d,'dHS'])
                if DF.loc[d,'LPpHS'] <= 1.0:
                    DF.loc[d,'pVrn2'] = 0
                else:
                    DF.loc[d,'pVrn2'] = calcpVrn2(DF.loc[d,'LPpHS'],Params.MaxpVrn2,Params.HfpVrn2HS)
                #DF.loc[d,'dpVrn2'] = DF.loc[d,'pVrn2'] - DF.loc[d-1,'pVrn2']
                #DF.loc[d,'PPdpVrn2'] = CalcdPPVrn(DF.loc[d,'Pp'], 0.0, DF.loc[d,'dpVrn2'],1.0)
            
            # Set todays Vrn expression values using deltas just calculated
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX'] + DF.loc[d,'dVrnX']
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1'] + DF.loc[d,'dBaseVrn1']
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] + DF.loc[d,'dMethColdVrn1'] 
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1'] + DF.loc[d,'dMethColdVrn1'] \
                                                          + DF.loc[d,'dBaseVrn1'] \
                                                          + DF.loc[d,'dVrnX'] 
            #DF.loc[d,'ColdVrn1'] = max(0.0,DF.loc[d-1,'ColdVrn1'] + DF.loc[d,'dColdVrn1'] - DF.loc[d,'dMethColdVrn1'])  
            #DF.loc[d,'PPpVrn2'] =  DF.loc[d-1,'PPpVrn2'] + DF.loc[d,'PPdpVrn2']
            DF.loc[d,'Vrn2'] =  max(0.0,DF.loc[d,'pVrn2'] - DF.loc[d,'Vrn1'])
                                    
        # Once Vernalisation complete, carry over vern expression values from yesterday
        else:
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1']
            DF.loc[d,'ColdVrn1'] = DF.loc[d-1,'ColdVrn1'] 
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] 
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1']
            DF.loc[d,'Vrn2'] = DF.loc[d-1,'Vrn2']
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX']
            
        #Set Haun stage variables
        if IsVernalised == False:
            DF.loc[d,'VSHS'] = DF.loc[d,'HS']
        else:
            DF.loc[d,'VSHS'] = DF.loc[d-1,'VSHS']
        if IsInduced == False:
            DF.loc[d,'FIHS'] = DF.loc[d,'HS']
        else: 
            DF.loc[d,'FIHS'] = DF.loc[d-1,'FIHS']
        if IsReproductive == False:
            DF.loc[d,'TSHS'] = DF.loc[d,'HS']
            DF.loc[d,'FLN'] = 2.86 + 1.1 * DF.loc[d,'TSHS']
        else:
            DF.loc[d,'TSHS'] = DF.loc[d-1,'TSHS']
            DF.loc[d,'FLN'] = DF.loc[d-1,'FLN']

        #work out if vernalisation is complete    
        #we get funny emergence time effects if we allow vernalisation to occur before emergence
        if (IsEmerged==True) and (DF.loc[d,'Vrn1'] >= Consts['VernalisationThreshold']) and (DF.loc[d,'Vrn2'] == 0) and (IsVernalised==False):
            IsVernalised  = True
            DF.loc[d,'Stage'] = 'Vern Sat'
        
        #Then work out Vrn3 expression
        if (IsVernalised == True) and (IsReproductive == False):
            DF.loc[d,'dVrn3'] = CalcdPPVrn(DF.loc[d,'Pp'], Params.BaseDVrn3, Params.MaxDVrn3, DF.loc[d,'dHS'])
        DF.loc[d,'Vrn3'] = min(1,DF.loc[d-1,'Vrn3'] + DF.loc[d,'dVrn3'])

        #Then work out phase progression based on Vrn expression
        if (DF.loc[d,'Vrn3'] >= 0.3) and (IsInduced == False):
            IsInduced = True;
        if (DF.loc[d,'Vrn3'] >=  1.0) and (IsReproductive == False):
            IsReproductive = True;
            DF.loc[d,'Stage'] = 'Term Spike'
        
        #Work out if Flag leaf has appeared.
        if DF.loc[d,'HS'] >= DF.loc[d,'FLN']:
            IsAtFlagLeaf = True    
            DF.loc[d,'Stage'] = 'FlagLeaf'
    
        #Add states to dataframe
        GerminationDay = False
        DF.loc[d,'IsGerminated'] = IsGerminated
        DF.loc[d,'IsEmerged'] = IsEmerged
        DF.loc[d,'IsMethalating'] = IsMethalating
        DF.loc[d,'IsVernalised'] = IsVernalised
        DF.loc[d,'IsReproductive'] = IsReproductive
        DF.loc[d,'IsAtFlagLeaf'] = IsAtFlagLeaf
    
        # Increment day
        d +=1
        
    if (Out == 'FLN') and (IsAtFlagLeaf == True):
        return DF.iloc[-1,:]['FLN']
    else:
        return DF
    
def CalcCultivarVrnCoeffs(inputs):
    """ Calculate VrnRate paremeters for genotype from given FLN observations
    Args:
        FLN_LV: Final Leaf Number under long day full vernalisation conditions
        FLN_LN: Final Leaf Number under long day un-vernalisation conditions
        FLN_SV: Final Leaf Number under short day full vernalisation conditions
        FLN_SN: Final Leaf Number under short day un-vernalisation conditions
        TreatmentTtDuration: ThermalTime Duration of vernalisation treatment
        TreatmentPTQ: The PTQ during vernalisation treatment,
        Tt: Vernalisation Treatment Temperature
        k: Vernalisation temperature response coefficient
        TtEmerge: Thermal time from sowing to emergence
        VernSatThreshold: The amount of Methalated Vrn1 that is required for vernilisation saturation to occur
        MethVrn1Threshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start
    Returns:
        Series with values assigned to each variable in CultivarParameterList 
    """
    #Set up Data Store
    data = pd.Series()
    
    data['maxLAR'] = inputs['maxLAR']
    data['minLAR'] = inputs['minLAR']
    data['PTQhf'] = inputs['PTQhf']
    
    # Haun stage equivelents from imbibing to Emergence, assume a PTQ of 1.0 suitable for before emergence
    data['EmergHS'] = inputs['TtEmerge_V']/(1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
    
    # Haun stage duration of vernalisation treatment period, assume a PTQ of 1.0 suitable for before emergence
    if inputs['TreatmentTtDuration'] <= inputs['TtEmerge_V']:
        data['VernTreatHS_L'] = inputs['TreatmentTtDuration']/\
                                (1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
        data['VernTreatHS_S'] = inputs['TreatmentTtDuration']/\
                                (1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
    else:
        data['EmergToEndTreatHS_L'] = (inputs['TreatmentTtDuration'] - inputs['TtEmerge_V'])/\
                                      (1/LARPTQ(inputs['TreatmentPTQ_L'],inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf'])) 
        data['EmergToEndTreatHS_S'] = (inputs['TreatmentTtDuration'] - inputs['TtEmerge_V'])/\
                                      (1/LARPTQ(inputs['TreatmentPTQ_S'],inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf'])) 
        data['VernTreatHS_L'] = data['EmergHS'] + data['EmergToEndTreatHS_L']
        data['VernTreatHS_S'] = data['EmergHS'] + data['EmergToEndTreatHS_S']
    
    # The soonest a plant may exhibit vern saturation
    MinVSHS = 1.1
    
    # Store FLN data
    data['FLN_LV']             = inputs['FLN_LV']
    data['FLN_LN']             = inputs['FLN_LN']
    data['FLN_SV']             = inputs['FLN_SV']
    data['FLN_SN']             = inputs['FLN_SN']
    
    # Calculate TSHS for each envronment set from FLNData
    data['TSHS_LV']            = TSHS( data['FLN_LV'])
    data['TSHS_LN']            = TSHS( data['FLN_LN'])
    data['TSHS_SV']            = TSHS( data['FLN_SV'])
    data['TSHS_SN']            = TSHS( data['FLN_SN'])
    
    # Haun stage duration from vernalisation saturation to terminal spikelet under long day conditions (MinHSVsTs)
                                 # Assume VS occurs at Competence HS under long Pp full vernalisation for varieties with FLN < 7.3 in these conditions
                                 # Assume maximum of 3, Data from Lincoln CE (CRWT153) showed varieties that harve a high TSHS hit VS ~3HS prior to TS under these conditions
    data['MinHSVsTs']          =  min(3,data['TSHS_LV'] - MinVSHS)
    
    
    # Photoperiod sensitivity (PPS)
                                 # the difference between TSHS at 8 and 16 h pp under 
                                 # full vernalisation.
    data['PPS']                =  max(0,data['TSHS_SV'] - data['TSHS_LV'])
    
    # Calculate VSHS for each environment from TSHS and photoperiod response
    data['VSHS_LV']            =  max(MinVSHS, data['TSHS_LV'] - data['MinHSVsTs'])
    data['VSHS_LN']            =  max(MinVSHS, data['TSHS_LN'] - data['MinHSVsTs']) 
    data['VSHS_SV']            =  max(MinVSHS, data['TSHS_SV'] - (data['MinHSVsTs'] + data['PPS']))
    data['VSHS_SN']            =  max(MinVSHS, data['TSHS_SN'] - (data['MinHSVsTs'] + data['PPS']))
    
    # Maximum delta for Upregulation of Vrn3 (MaxDVrn3)
                                 # Occurs under long Pp conditions
                                 # Assuming Vrn3 increases from 0 - VernSatThreshold between VS to TS and this takes 
                                 # 3 HS under long Pp conditions.
    data['MaxDVrn3']           = inputs['VernalisationThreshold'] / data['MinHSVsTs']
    
    # Base delta for upredulation of Vrn3 (BaseDVrn3) 
                                 # Occurs under short Pp conditions
                                 # Assuming Vrn3 infreases from 0 - VernSatThreshold from VS to TS 
                                 # and this take 3 HS plus the additional HS from short Pp delay.
    data['BaseDVrn3']          = inputs['VernalisationThreshold'] / (data['MinHSVsTs'] + data['PPS'])
    
    # Base delta for upregulation of Vrn1 (BaseDVrn1) 
                                 # Occurs under non vernalising conditions
                                 # Assuming Vernalisation saturation occurs when Vrn1 == VernSatThreshold  
                                 # under short Pp when no Vrn2 or VrnX are expressed.
                                 # Need to include time from imbib to emerge in duration 
    data['BaseDVrn1']          = inputs['VernalisationThreshold'] / (data['VSHS_SN'] + data['EmergHS'])
    
    # Use LV treatment to calculate cold response as this is when vrn2 is expressed so shows the greatest Vrn1 requirement from cold
    # BaseVern1 expressed at vernalisation saturation
    data['BaseVrn1AtVSHS_SV']  =  (data['VSHS_SV']+data['EmergHS']) * data['BaseDVrn1']
    
    # Methalated Cold upredulated Vrn1 expression at VSHS
                                 # Subtract out BasedVrn1 expression up to transition
    data['MethColdVern1AtTrans_SV'] =  max(0,inputs['VernalisationThreshold'] - data['BaseVrn1AtVSHS_SV'])
    
    # ColdVrn1AtVSHS_SV is the amount of Vrn1 upregulation caused by cold that would be required to give the 
                                 #required MethVern1AtVSHS_SV
    data['ColdVrn1AtVSHS_SV']  = inputs['MethalationThreshold'] + data['MethColdVern1AtTrans_SV']
    
    # The haun stage duration over which cold temperatures will have an effect.  Will be the minimum of the cold 
    # treatment duration (VernTreatHS) and the HS when vernalisation occurs as cold exposure after this is irrelevent
    data['HSColdEffect']       = min(data['VSHS_SV']+data['EmergHS'], data['VernTreatHS_S']);
    
    # Cold induced delta upregulation of Vrn1 at treatment temperature ('DVrn1@Tt')
                                 # Methalation of Cold upregulated Vrn1 occurs when 
                                 # ColdUpRegVrn1 > Vrn1Target
                                 # Vrn1Target will be 1.0 in 8 hour conditions
                                 # so ColdUpRegVrn1 at transition will be:
                                 # 1.0 - BaseVern1@Meth + MethColdVrn1@Trans.
                                 # divide by treatment HS duration to give rate
    data['DVrn1AtVrnTreatTemp'] = data['ColdVrn1AtVSHS_SV']/data['HSColdEffect']
    
    # Maximum upregulation delta Vrn1 (MUdVrn1)
                                 # The rate of dVrn1/HS at 0oC.  Calculate by rearanging UdVrn1 equation
                                 # UdVrn1 = MUdVrn1 * np.exp(k*Tt) as UdVrn1@Tt is known 
    data['MaxDVrn1']           = data['DVrn1AtVrnTreatTemp'] / np.exp(inputs['k'] * inputs['VrnTreatTemp']) 
    
    # Vernalisation photoperiod sensitivity, how much does exposure to long photoperiod delay (negative) or acellerate 
                                # (positive) the timing of VS
    data['VPPS']               = 1-((data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    
    # for genotypes that have a negative VPPS we calculate Vrn2 expression parameters 
                                # MaxIVrn2 is the maximum initial potential Vrn2 expressed at 8h Pp the day of emergence (assuming rapid up regulation at this time)
                                # MaxDVrn2 is the rate that potential Vrn2 expression increases with HS as the plat develops
                                # Need to calculate potential Vrn2 (pVrn2) at VSHS for long Pp with and without Vrn and use
                                # these with linear regression to estimate MaxIVrn2 and MaxDVrn2
    data['BaseVrn1AtVSHS_LV']  = (data['VSHS_LV']+data['EmergHS']) * data['BaseDVrn1']
    data['pVrn2AtVSHS_LV']     = data['BaseVrn1AtVSHS_LV']#0.95
    data['pVrn2AtVSHS_LN']     = (data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1']
    
    Vrn2Fit                    = fitVrn2Coeffs(data['VPPS'],
                                                data['BaseDVrn1'],
                                                data['pVrn2AtVSHS_LV'],
                                                data['pVrn2AtVSHS_LN'],
                                                data['VSHS_LV'],
                                                data['VSHS_LN'])
    data['MaxpVrn2']  = Vrn2Fit[0]
    data['HfpVrn2HS']   = Vrn2Fit[1]
      
    
     # Maximum delta upregulation for VrnX (MaxDVrnX)
                                 # This is an unidentified gene that accellerates vernalisation under long day conditions.
                                 # Only relevent for varieties that have MaxDVrn2 less than 1 (i.e no short day vernalisation response from Vrn2)
                                 # Estimate as the rate of additional Vrn1 expression needed to get to VSHS_LN over and above BaseVrn1 expression a this point
    data['BaseVrn1AtVSHS_LN']   = min(inputs['VernalisationThreshold'],(data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    data['MaxDVrnX']            = (inputs['VernalisationThreshold']-data['BaseVrn1AtVSHS_LN'])/(data['VSHS_LN'])
    
    return data     

""" Calculate Vrn2 rate parameters
Args:
    VPPS: Vernalisation Photo Period Sensitivity
    pVrn2AtVSHS_LV: The potential Vrn2 expressoin at the time of Vern saturation under LV conditions
    pVrn2AtVSHS_LN: The potential Vrn2 expressoin at the time of Vern saturation under LN conditions
    VSHS_LV: Vernalisation Saturation Haun Stage under LV conditions
    VSHS_LN: Vernalisation Saturation Haun Stage under LN conditions
Returns:
    tupple (a,b) where a is the initial pVrn2 expressed on day of emergence and b is the increase in pVrn2/HS as the plant develops
"""
def calcVrn2Coeffs(VPPS,BaseDVrn1,pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN,EmergeHS):
    a = 0
    b = 0
    if (VPPS < 0) and (BaseDVrn1 < 0.4):
        try:
            b = (pVrn2AtVSHS_LN-pVrn2AtVSHS_LV)/(VSHS_LN-VSHS_LV)
            a = 1-(VSHS_LV+EmergeHS)*b
            if a<0:
                a,b = forceInt(pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN)
        except:
            a,b = forceInt(pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN)
    b =  min(b,BaseDVrn1)
    return (a,b)

""" Recalculate Vrn2 rate parameters, forcing intercept to 0.1 where it is less than this
Args:
    VPPS: Vernalisation Photo Period Sensitivity
    pVrn2AtVSHS_LV: The potential Vrn2 expressoin at the time of Vern saturation under LV conditions
    pVrn2AtVSHS_LN: The potential Vrn2 expressoin at the time of Vern saturation under LN conditions
    VSHS_LV: Vernalisation Saturation Haun Stage under LV conditions
    VSHS_LN: Vernalisation Saturation Haun Stage under LN conditions
Returns:
    tupple (a,b) where a is the initial pVrn2 expressed on day of emergence and b is the increase in pVrn2/HS as the plant develops
"""
def forceInt(pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN):
    a = 0
    b = pVrn2AtVSHS_LV/VSHS_LV
    return (a,b)

def calcpVrn2(HS,MaxpVrn2,HfpVrn2):
    return (MaxpVrn2*(HS-1))/(HfpVrn2+(HS-1))

def fitVrn2Coeffs(VPPS,BaseDVrn1,pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN):
    if (VPPS < 0) and (BaseDVrn1<0.4):
        xdat = [1,VSHS_LV,VSHS_LN]
        ydat = [0,pVrn2AtVSHS_LV,pVrn2AtVSHS_LN]
        return curve_fit(calcpVrn2,xdat,ydat,bounds=(0,[10,10]))[0]
    else:
        return [0,1]
        

Overwriting C:\Anaconda\Lib\CAMP.py
